<br>

# Introdução


In [ ]:
import time
import pandas as pd

from gecko import *
import requests
from paths import *
from pathlib import Path
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.webdriver.firefox.service import Service as FirefoxService


<br>

## Driver


In [ ]:
# Services
gecko_path = get_path_geckodriver(driver_path)

# Logs
logs_filepath = logs_path / 'geckodriver.log'

# Services
service = FirefoxService(executable_path=gecko_path, log_path=logs_filepath)

# Options
options = FirefoxOptions()
options.headless = False
options.set_preference('intl.accept_languages', 'pt-BR, pt')

# Driver
driver = webdriver.Firefox(service=service, options=options)
driver.maximize_window()


In [ ]:
# Add-ons Xpath
xpath_path = adds_path / 'xpath.xpi'
xpath_path = xpath_path.absolute().resolve()
if not xpath_path.is_file():
    url = 'https://addons.mozilla.org/firefox/downloads/file/3588871/xpath_finder-1.0.2-fx.xpi'
    r = requests.get(url, timeout=60, verify=False)

    with open(xpath_path, 'wb') as f:
        f.write(r.content)

driver.install_addon(str(xpath_path), temporary=True)


<br>

## Functions

In [ ]:
# Voltar
def click_back():
    content_xpath = driver.find_element(By.XPATH, "//*[@id='content']")
    back_xpath = content_xpath.find_element(
        By.XPATH, ".//*[@class='item folder folder-parent']"
    )
    back_xpath.click()
    time.sleep(1)
    return 0


In [ ]:
# # Conteúdo Principal
# content_xpath = driver.find_element(By.XPATH, "//*[@id='content']")
# content_xpath

# # Lista
# list_folders = content_xpath.find_elements(
#     By.XPATH, ".//*[@class='item folder']"
# )
# list_folders = list_folders[0:3]
# list_folders


In [ ]:
def list_folders_h5ai():
    """
    Lista as Pastas

    :return: _description_
    :rtype: _type_
    """

    # Conteúdo Principal
    content_xpath = driver.find_element(By.XPATH, "//*[@id='content']")

    # Lista
    list_folders = content_xpath.find_elements(
        By.XPATH, ".//*[@class='item folder']"
    )

    list_data = []
    for folder in list_folders:
        # URL
        href_xpath = folder.find_element(By.XPATH, ".//a")
        href_value = href_xpath.get_attribute('href')
        # print(href_value)

        # Tipo
        icon_xpath = folder.find_element(
            By.XPATH, ".//a//span[@class='icon square']//img"
        )
        icon_value = icon_xpath.get_attribute('alt')
        # print(icon_value)

        # Nome
        label_xpath = folder.find_element(
            By.XPATH, ".//a//span[@class='label']"
        )
        label_value = label_xpath.text
        # print(label_value)

        # Data
        date_xpath = folder.find_element(By.XPATH, ".//a//span[@class='date']")
        date_value = date_xpath.text
        # print(date_value)

        # Size
        size_xpath = folder.find_element(By.XPATH, ".//a//span[@class='size']")
        size_value = size_xpath.text
        # print(size_value)

        #
        dict_data = {
            'url': href_value,
            'type': icon_value,
            'name': label_value,
            'date': date_value,
            'size': size_value,
        }
        # print(dict_data)
        list_data.append(dict_data)

    return list_data


In [ ]:
def list_itens_h5ai():
    # Conteúdo Principal
    content_xpath = driver.find_element(By.XPATH, "//*[@id='content']")

    # Lista
    list_itens = content_xpath.find_elements(
        By.XPATH, ".//*[@class='item file']"
    )

    list_data = []
    for folder in list_itens:
        # URL
        href_xpath = folder.find_element(By.XPATH, ".//a")
        href_value = href_xpath.get_attribute('href')
        # print(href_value)

        # Tipo
        icon_xpath = folder.find_element(
            By.XPATH, ".//a//span[@class='icon square']//img"
        )
        icon_value = icon_xpath.get_attribute('alt')
        # print(icon_value)

        # Nome
        label_xpath = folder.find_element(
            By.XPATH, ".//a//span[@class='label']"
        )
        label_value = label_xpath.text
        # print(label_value)

        # Data
        date_xpath = folder.find_element(By.XPATH, ".//a//span[@class='date']")
        date_value = date_xpath.text
        # print(date_value)

        # Size
        size_xpath = folder.find_element(By.XPATH, ".//a//span[@class='size']")
        size_value = size_xpath.text
        # print(size_value)

        #
        dict_data = {
            'url': href_value,
            'type': icon_value,
            'name': label_value,
            'date': date_value,
            'size': size_value,
        }
        # print(dict_data)
        list_data.append(dict_data)
    return list_data


In [ ]:
def find_folder_by_name(name, click=True):
    # Conteúdo Principal
    content_xpath = driver.find_element(By.XPATH, "//*[@id='content']")

    try:
        foldername_xpath = content_xpath.find_element(
            By.XPATH, f".//span[text()='{name}']"
        )
        if click:
            foldername_xpath.click()
            time.sleep(1)
        return foldername_xpath        
    except Exception as e:
        print(e)
        raise ValueError('A very specific bad thing happened')

#find_folder_by_name('ADAMANTINA')


<br>

# Run

In [ ]:
URL = 'https://geo.fbds.org.br/SP/'
#URL = 'https://geo.fbds.org.br/SP/ADAMANTINA/'
driver.get(URL)
time.sleep(5)

In [ ]:
list_all_files = []
list_all_folders = []


def loop():
    """
    Ponto inicio

    Lista todos os arquivos e junta!
    Define N Pastas

    Lista todas as pastas
    Se N pastas > 0, entra na pasta....
    Se N pastas < 0, 
    """

    # Lista arquivos
    list_files = list_itens_h5ai()

    # Se tem Arquivos, junta na lista geral
    if len(list_files) > 0:
        list_all_files.extend(list_files)
        print(f' > Lista geral está com: {len(list_all_files)} arquivos')
    else:
        print(f' > Pasta sem arquivos')

    # Lista Pastas
    list_folders = list_folders_h5ai()
    list_all_folders.extend(list_folders)    
    if len(list_folders) > 0:
        for folder in list_folders:
            # Parâmetros
            folder_name = folder['name']            
            print(f'{folder_name}')

            # Entra na Pasta
            find_folder_by_name(folder_name)

            # Recursive
            loop()

            # Sai da Pasta
            click_back()
            time.sleep(3)
    else:
        print(f' > Pasta sem subpastas...')

    #return list_all_files, list_all_folders

In [ ]:
#list_all_files, list_all_folders = 
loop()

In [ ]:
df = pd.DataFrame(list_all_files)
df.to_csv(
    input_path / 'ddd.csv',
    encoding='utf-8',
    index=False
)

In [ ]:
# Close Connection
driver.quit()
